## Functions

In [1]:
def compute_players_stats_all_matches(match, matches):
    p1 = match["server1"]
    p2 = match["server2"]
    
    serve_count = {}
    return_count = {}
    service_game_count = {}
    return_game_count = {}
    sets_count = {}
    tiebreaks_count = {}
    match_count = {}
    
    all_player_count = {}
    all_player_count["serves won"] = 0
    all_player_count["serves total"] = 0
    all_player_count["service games won"] = 0
    all_player_count["service games total"] = 0

    for index, match_ in matches.iterrows():
        player1 = match_["server1"]
        player2 = match_["server2"]
        
        if player1 not in serve_count:
            serve_count[player1] = [0,0]
            return_count[player1] = [0,0]
            service_game_count[player1] = [0,0]
            return_game_count[player1] = [0,0]
            sets_count[player1] = [0,0]
            tiebreaks_count[player1] = [0,0]
            match_count[player1] = 0
        if player2 not in serve_count:
            serve_count[player2] = [0,0]
            return_count[player2] = [0,0]
            service_game_count[player2] = [0,0]
            return_game_count[player2] = [0,0]
            sets_count[player2] = [0,0]
            tiebreaks_count[player2] = [0,0]
            match_count[player2] = 0
        
        serve_count[player1][0] += match_["p1 serves won"]
        serve_count[player2][0] += match_["p2 serves won"]
        serve_count[player1][1] += match_["p1 serves total"]
        serve_count[player2][1] += match_["p2 serves total"]
        all_player_count["serves won"] += match_["p1 serves won"] + match_["p2 serves won"]
        all_player_count["serves total"] += match_["p1 serves total"] + match_["p2 serves total"]


        return_count[player1][0] += match_["p2 serves total"] - match_["p2 serves won"]
        return_count[player2][0] += match_["p1 serves total"] - match_["p1 serves won"]
        return_count[player1][1] += match_["p2 serves total"]
        return_count[player2][1] += match_["p1 serves total"]

        
        service_game_count[player1][0] += match_["p1 service games won"]
        service_game_count[player2][0] += match_["p2 service games won"]
        service_game_count[player1][1] += match_["p1 service games total"]
        service_game_count[player2][1] += match_["p2 service games total"]
        all_player_count["service games won"] += match_["p1 service games won"] + match_["p2 service games won"]
        all_player_count["service games total"] += match_["p1 service games total"] + match_["p2 service games total"]

        
        return_game_count[player1][0] += match_["p2 service games total"] - match_["p2 service games won"]
        return_game_count[player2][0] += match_["p1 service games total"] - match_["p1 service games won"]
        return_game_count[player1][1] += match_["p2 service games total"]
        return_game_count[player2][1] += match_["p1 service games total"]

        
        sets_count[player1][0] += match_["p1 sets won"]
        sets_count[player2][0] += match_["p2 sets won"]
        sets_count[player1][1] += match_["sets total"]
        sets_count[player2][1] += match_["sets total"]

        
        tiebreaks_count[player1][0] += match_["p1 tiebreaks won"]
        tiebreaks_count[player2][0] += match_["p2 tiebreaks won"]
        tiebreaks_count[player1][1] += match_["p1 tiebreaks won"] + match_["p2 tiebreaks won"]
        tiebreaks_count[player2][1] += match_["p1 tiebreaks won"] + match_["p2 tiebreaks won"]

        
        match_count[player1] += 1
        match_count[player2] += 1
    
    
    return serve_count, return_count, service_game_count, return_game_count, sets_count, tiebreaks_count, match_count, all_player_count

In [2]:
def calculate_beta_binomial_estimators_past_year(match, matches, match_count_threshold):
    p1 = match["server1"]
    p2 = match["server2"]
    
    date = match["date"]
    one_year_prior = date - relativedelta(years=1)
    relevant_matches = matches[(matches["date"] < date) & (matches["date"] >= one_year_prior)]
    
    serve_count, return_count, service_game_count, return_game_count, sets_count, tiebreaks_count, match_count, all_player_count = compute_players_stats_all_matches(match, relevant_matches)
    
    serve_wr = []
    return_wr = []
    service_game_wr = []
    return_game_wr = []
    sets_wr = []
    tiebreaks_wr = []
    serve_total = []
    return_total = []
    service_game_total = []
    return_game_total = []
    sets_total = []
    tiebreaks_total = []
    for player in match_count:
        if player in match_count and match_count[player] > match_count_threshold:
            serve_wr.append(serve_count[player][0] / serve_count[player][1])
            serve_total.append(serve_count[player][1])
            
            return_wr.append(return_count[player][0] / return_count[player][1])
            return_total.append(return_count[player][1])
            
            service_game_wr.append(service_game_count[player][0] / service_game_count[player][1])
            service_game_total.append(service_game_count[player][1])
            
            return_game_wr.append(return_game_count[player][0] / return_game_count[player][1])
            return_game_total.append(return_game_count[player][1])
            
            sets_wr.append(sets_count[player][0] / sets_count[player][1])
            sets_total.append(sets_count[player][1])
            
            if tiebreaks_count[player][1] > 0:
                tiebreaks_wr.append(tiebreaks_count[player][0] / tiebreaks_count[player][1])
                tiebreaks_total.append(tiebreaks_count[player][1])
    
    serve_alpha, serve_beta = float("nan"), float("nan")
    return_alpha, return_beta = float("nan"), float("nan")
    service_game_alpha, service_game_beta = float("nan"), float("nan")
    return_game_alpha, return_game_beta = float("nan"), float("nan")
    sets_alpha, sets_beta = float("nan"), float("nan")
    tiebreaks_alpha, tiebreaks_beta = float("nan"), float("nan")
    
    if len(serve_wr) > 2:
        serve_wr = [(x*(len(serve_wr) - 1) + 0.5) / len(serve_wr) for x in serve_wr]
        serve_alpha, serve_beta, _1, _2 = beta.fit(serve_wr, floc=0, fscale=1)
        
    if len(return_wr) > 2:
        return_wr = [(x*(len(return_wr) - 1) + 0.5) / len(return_wr) for x in return_wr]
        return_alpha, return_beta, _1, _2 = beta.fit(return_wr, floc=0, fscale=1)
        
    if len(service_game_wr) > 2:
        service_game_wr = [(x*(len(service_game_wr) - 1) + 0.5) / len(service_game_wr) for x in service_game_wr]
        service_game_alpha, service_game_beta, _1, _2 = beta.fit(service_game_wr, floc=0, fscale=1)
        
    if len(return_game_wr) > 2:
        return_game_wr = [(x*(len(return_game_wr) - 1) + 0.5) / len(return_game_wr) for x in return_game_wr]
        return_game_alpha, return_game_beta, _1, _2 = beta.fit(return_game_wr, floc=0, fscale=1)
        
    if len(sets_wr) > 2:
        sets_wr = [(x*(len(sets_wr) - 1) + 0.5) / len(sets_wr) for x in sets_wr]
        sets_alpha, sets_beta, _1, _2 = beta.fit(sets_wr, floc=0, fscale=1)
        
    if len(tiebreaks_wr) > 2:
        try:
            tiebreaks_wr = [(x*(len(tiebreaks_wr) - 1) + 0.5) / len(tiebreaks_wr) for x in tiebreaks_wr]
            tiebreaks_alpha, tiebreaks_beta, _1, _2 = beta.fit(tiebreaks_wr, floc=0, fscale=1)
        except Exception:
            pass
    
    
    a = float("nan")
    b = float("nan")
    c = float("nan")
    d = float("nan")
    e = float("nan")
    f = float("nan")
    g = float("nan")
    h = float("nan")
    i = float("nan")
    j = float("nan")
    k = float("nan")
    l = float("nan")
    m = serve_alpha
    n = serve_beta
    o = return_alpha
    p = return_beta
    q = service_game_alpha
    r = service_game_beta
    s = return_game_alpha
    t = return_game_beta
    u = sets_alpha
    v = sets_beta
    w = tiebreaks_alpha
    x = tiebreaks_beta
    
    # p1 beta shrunken values
    if p1 in serve_count and serve_count[p1][1] > 0:
        a = (serve_count[p1][0] + serve_alpha) / (serve_count[p1][1] + serve_alpha + serve_beta)
    if p1 in return_count and return_count[p1][1] > 0:
        b = (return_count[p1][0] + return_alpha) / (return_count[p1][1] + return_alpha + return_beta)
    if p1 in service_game_count and service_game_count[p1][1] > 0:
        c = (service_game_count[p1][0] + service_game_alpha) / (service_game_count[p1][1] + service_game_alpha + service_game_beta)
    if p1 in return_game_count and return_game_count[p1][1] > 0:
        d = (return_game_count[p1][0] + return_game_alpha) / (return_game_count[p1][1] + return_game_alpha + return_game_beta)
    if p1 in sets_count and sets_count[p1][1] > 0:
        e = (sets_count[p1][0] + sets_alpha) / (sets_count[p1][1] + sets_alpha + sets_beta)
    if p1 in tiebreaks_count and tiebreaks_count[p1][1] > 0:
        f = (tiebreaks_count[p1][0] + tiebreaks_alpha) / (tiebreaks_count[p1][1] + tiebreaks_alpha + tiebreaks_beta)
    
    # p2 beta shrunken values
    if p2 in serve_count and serve_count[p2][1] > 0:
        g = (serve_count[p2][0] + serve_alpha) / (serve_count[p2][1] + serve_alpha + serve_beta)
    if p2 in return_count and return_count[p2][1] > 0:
        h = (return_count[p2][0] + return_alpha) / (return_count[p2][1] + return_alpha + return_beta)
    if p2 in service_game_count and service_game_count[p2][1] > 0:
        i = (service_game_count[p2][0] + service_game_alpha) / (service_game_count[p2][1] + service_game_alpha + service_game_beta)
    if p2 in return_game_count and return_game_count[p2][1] > 0:
        j = (return_game_count[p2][0] + return_game_alpha) / (return_game_count[p2][1] + return_game_alpha + return_game_beta)
    if p2 in sets_count and sets_count[p2][1] > 0:
        k = (sets_count[p2][0] + sets_alpha) / (sets_count[p2][1] + sets_alpha + sets_beta)
    if p2 in tiebreaks_count and tiebreaks_count[p2][1] > 0:
        l = (tiebreaks_count[p2][0] + tiebreaks_alpha) / (tiebreaks_count[p2][1] + tiebreaks_alpha + tiebreaks_beta)
        
    return a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x
    

In [3]:
from scipy.special import loggamma
from scipy.optimize import minimize

def calculate_mu_i(X, beta):
    return 1 / (1 + np.exp(-(np.dot(X, beta))))

def calculate_neg_loglikelihood(params, y, X):
    beta = params[0:-1]
    phi = params[-1]
    mu_i = calculate_mu_i(X, beta)
    ll = loggamma(phi) - loggamma(mu_i*phi) - loggamma((1 - mu_i)*phi) + (mu_i*phi - 1)*np.log(y) + ((1-mu_i)*phi-1)*np.log(1-y)
    
    return -np.sum(ll)


def calculate_optimal_beta_and_phi(rate, total, alpha0, beta0):
    # initialize parameters for optimization
    ones = [1] * len(total)
    X = np.array([total, ones]).T
    y = np.array(rate)
    b0 = 0
    
    t = alpha0 / (alpha0 + beta0)
    b1 = np.log(t / (1-t))
    
    
    phi = alpha0 + beta0
    params = np.array([b0,b1,phi])
    
    res = minimize(calculate_neg_loglikelihood, x0=params, args=(y,X), bounds=[(None,None), 
                                                         (None,None), (0,None)], options={'ftol': 1e-06, 'gtol': 1e-06})
    
    if res.success:
        return res.x
    else:
        return [float("nan"), float("nan"), float("nan")]
    
    

In [4]:
def calculate_beta_binomial_regression_estimators_past_year(match, matches, match_count_threshold):
    p1 = match["server1"]
    p2 = match["server2"]
    
    date = match["date"]
    one_year_prior = date - relativedelta(years=1)
    relevant_matches = matches[(matches["date"] < date) & (matches["date"] >= one_year_prior)]
    
    serve_count, return_count, service_game_count, return_game_count, sets_count, tiebreaks_count, match_count, all_player_count = compute_players_stats_all_matches(match, relevant_matches)
    
    serve_wr = []
    return_wr = []
    service_game_wr = []
    return_game_wr = []
    sets_wr = []
    tiebreaks_wr = []
    serve_total = []
    return_total = []
    service_game_total = []
    return_game_total = []
    sets_total = []
    tiebreaks_total = []
    for player in match_count:
        if player in match_count and match_count[player] > match_count_threshold:
            serve_wr.append(serve_count[player][0] / serve_count[player][1])
            serve_total.append(np.log(np.log(serve_count[player][1])))
            
            return_wr.append(return_count[player][0] / return_count[player][1])
            return_total.append(np.log(np.log(return_count[player][1])))
            
            service_game_wr.append(service_game_count[player][0] / service_game_count[player][1])
            service_game_total.append(np.log(np.log(service_game_count[player][1])))
            
            return_game_wr.append(return_game_count[player][0] / return_game_count[player][1])
            return_game_total.append(np.log(np.log(return_game_count[player][1])))
            
            sets_wr.append(sets_count[player][0] / sets_count[player][1])
            sets_total.append(np.log(np.log(sets_count[player][1])))
            if tiebreaks_count[player][1] > 0:
                tiebreaks_wr.append(tiebreaks_count[player][0] / tiebreaks_count[player][1])
                tiebreaks_total.append(np.log(np.log(tiebreaks_count[player][1])))
    
    serve_beta0, serve_beta1, serve_phi = float("nan"), float("nan"), float("nan")
    return_beta0, return_beta1, return_phi = float("nan"), float("nan"), float("nan")
    service_game_beta0, service_game_beta1, service_game_phi = float("nan"), float("nan"), float("nan")
    return_game_beta0, return_game_beta1, return_game_phi = float("nan"), float("nan"), float("nan")
    sets_beta0, sets_beta1, sets_phi = float("nan"), float("nan"), float("nan")
    tiebreaks_beta0, tiebreaks_beta1, tiebreaks_phi = float("nan"), float("nan"), float("nan")
    
    serve_alpha, serve_beta = match["serve alpha"], match["serve beta"]
    return_alpha, return_beta = match["return alpha"], match["return beta"]
    service_game_alpha, service_game_beta = match["service game alpha"], match["service game beta"]
    return_game_alpha, return_game_beta = match["return game alpha"], match["return game beta"]
    sets_alpha, sets_beta = match["set alpha"], match["set beta"]
    tiebreaks_alpha, tiebreaks_beta = match["tiebreak alpha"], match["tiebreak beta"]
    
    if len(serve_wr) > 2:
        serve_wr = [(x*(len(serve_wr) - 1) + 0.5) / len(serve_wr) for x in serve_wr]
        serve_beta0, serve_beta1, serve_phi = tuple(calculate_optimal_beta_and_phi(serve_wr, serve_total, serve_alpha, serve_beta))
        
    if len(return_wr) > 2:
        return_wr = [(x*(len(return_wr) - 1) + 0.5) / len(return_wr) for x in return_wr]
        return_beta0, return_beta1, return_phi = tuple(calculate_optimal_beta_and_phi(return_wr, return_total, return_alpha, return_beta))
        
    if len(service_game_wr) > 2:
        service_game_wr = [(x*(len(service_game_wr) - 1) + 0.5) / len(service_game_wr) for x in service_game_wr]
        service_game_beta0, service_game_beta1, service_game_phi = tuple(calculate_optimal_beta_and_phi(service_game_wr, service_game_total, service_game_alpha, service_game_beta))
        
    if len(return_game_wr) > 2:
        return_game_wr = [(x*(len(return_game_wr) - 1) + 0.5) / len(return_game_wr) for x in return_game_wr]
        return_game_beta0, return_game_beta1, return_game_phi = tuple(calculate_optimal_beta_and_phi(return_game_wr, return_game_total, return_game_alpha, return_game_beta))
        
    if len(sets_wr) > 2:
        sets_wr = [(x*(len(sets_wr) - 1) + 0.5) / len(sets_wr) for x in sets_wr]
        sets_beta0, sets_beta1, sets_phi = tuple(calculate_optimal_beta_and_phi(sets_wr, sets_total, sets_alpha, sets_beta))
        
    if len(tiebreaks_wr) > 2:
        try:
            tiebreaks_wr = [(x*(len(tiebreaks_wr) - 1) + 0.5) / len(tiebreaks_wr) for x in tiebreaks_wr]
            tiebreaks_beta0, tiebreaks_beta1, tiebreaks_phi = tuple(calculate_optimal_beta_and_phi(tiebreaks_wr, tiebreaks_total, tiebreaks_alpha, tiebreaks_beta))
        except Exception as e:
            pass
    
    
    a = float("nan")
    b = float("nan")
    c = float("nan")
    d = float("nan")
    e = float("nan")
    f = float("nan")
    g = float("nan")
    h = float("nan")
    i = float("nan")
    j = float("nan")
    k = float("nan")
    l = float("nan")
    serve_alpha, serve_beta = float("nan"), float("nan")
    return_alpha, return_beta = float("nan"), float("nan")
    service_game_alpha, service_game_beta = float("nan"), float("nan")
    return_game_alpha, return_game_beta = float("nan"), float("nan")
    sets_alpha, sets_beta = float("nan"), float("nan")
    tiebreaks_alpha, tiebreaks_beta = float("nan"), float("nan")
    
    # p1 beta shrunken values
    if p1 in serve_count and serve_count[p1][1] > 0:
        serve_beta = np.array([serve_beta0, serve_beta1])
        X = np.array([np.log(np.log(serve_count[p1][1])), 1])
        mu_i = calculate_mu_i(X, serve_beta)

        serve_alpha = mu_i * serve_phi
        serve_beta = (1 - mu_i) * serve_phi
        a = (serve_count[p1][0] + serve_alpha) / (serve_count[p1][1] + serve_alpha + serve_beta)
    if p1 in return_count and return_count[p1][1] > 0:
        return_beta = np.array([return_beta0, return_beta1])
        X = np.array([np.log(np.log(return_count[p1][1])), 1])
        mu_i = calculate_mu_i(X, return_beta)
        
        return_alpha = mu_i * return_phi
        return_beta = (1 - mu_i) * return_phi
        b = (return_count[p1][0] + return_alpha) / (return_count[p1][1] + return_alpha + return_beta)
    if p1 in service_game_count and service_game_count[p1][1] > 0:
        service_game_beta = np.array([service_game_beta0, service_game_beta1])
        X = np.array([np.log(np.log(service_game_count[p1][1])), 1])
        mu_i = calculate_mu_i(X, service_game_beta)
        
        service_game_alpha = mu_i * service_game_phi
        service_game_beta = (1 - mu_i) * service_game_phi
        c = (service_game_count[p1][0] + service_game_alpha) / (service_game_count[p1][1] + service_game_alpha + service_game_beta)
    if p1 in return_game_count and return_game_count[p1][1] > 0:
        return_game_beta = np.array([return_game_beta0, return_game_beta1])
        X = np.array([np.log(np.log(return_game_count[p1][1])), 1])
        mu_i = calculate_mu_i(X, return_game_beta)
        
        return_game_alpha = mu_i * return_game_phi
        return_game_beta = (1 - mu_i) * return_game_phi
        d = (return_game_count[p1][0] + return_game_alpha) / (return_game_count[p1][1] + return_game_alpha + return_game_beta)
    
    if p1 in sets_count and sets_count[p1][1] > 0:
        sets_beta = np.array([sets_beta0, sets_beta1])
        X = np.array([np.log(np.log(sets_count[p1][1])), 1])
        mu_i = calculate_mu_i(X, sets_beta)
        
        sets_alpha = mu_i * sets_phi
        sets_beta = (1 - mu_i) * sets_phi
        e = (sets_count[p1][0] + sets_alpha) / (sets_count[p1][1] + sets_alpha + sets_beta)
    if p1 in tiebreaks_count and tiebreaks_count[p1][1] > 0:
        tiebreaks_beta = np.array([tiebreaks_beta0, tiebreaks_beta1])
        X = np.array([np.log(np.log(tiebreaks_count[p1][1])), 1])
        mu_i = calculate_mu_i(X, tiebreaks_beta)
        
        tiebreaks_alpha = mu_i * tiebreaks_phi
        tiebreaks_beta = (1 - mu_i) * tiebreaks_phi
        f = (tiebreaks_count[p1][0] + tiebreaks_alpha) / (tiebreaks_count[p1][1] + tiebreaks_alpha + tiebreaks_beta)
    
    # p2 beta shrunken values
    if p2 in serve_count and serve_count[p2][1] > 0:
        serve_beta = np.array([serve_beta0, serve_beta1])
        X = np.array([np.log(np.log(serve_count[p2][1])), 1])
        mu_i = calculate_mu_i(X, serve_beta)
        
        serve_alpha = mu_i * serve_phi
        serve_beta = (1 - mu_i) * serve_phi
        g = (serve_count[p2][0] + serve_alpha) / (serve_count[p2][1] + serve_alpha + serve_beta)
    if p2 in return_count and return_count[p2][1] > 0:
        return_beta = np.array([return_beta0, return_beta1])
        X = np.array([np.log(np.log(return_count[p2][1])), 1])
        mu_i = calculate_mu_i(X, return_beta)
        
        return_alpha = mu_i * return_phi
        return_beta = (1 - mu_i) * return_phi
        h = (return_count[p2][0] + return_alpha) / (return_count[p2][1] + return_alpha + return_beta)
    if p2 in service_game_count and service_game_count[p2][1] > 0:
        service_game_beta = np.array([service_game_beta0, service_game_beta1])
        X = np.array([np.log(np.log(service_game_count[p2][1])), 1])
        mu_i = calculate_mu_i(X, service_game_beta)
        
        service_game_alpha = mu_i * service_game_phi
        service_game_beta = (1 - mu_i) * service_game_phi
        i = (service_game_count[p2][0] + service_game_alpha) / (service_game_count[p2][1] + service_game_alpha + service_game_beta)
    if p2 in return_game_count and return_game_count[p2][1] > 0:
        return_game_beta = np.array([return_game_beta0, return_game_beta1])
        X = np.array([np.log(np.log(return_game_count[p2][1])), 1])
        mu_i = calculate_mu_i(X, return_game_beta)
        
        return_game_alpha = mu_i * return_game_phi
        return_game_beta = (1 - mu_i) * return_game_phi
        j = (return_game_count[p2][0] + return_game_alpha) / (return_game_count[p2][1] + return_game_alpha + return_game_beta)
    if p2 in sets_count and sets_count[p2][1] > 0:
        sets_beta = np.array([sets_beta0, sets_beta1])
        X = np.array([np.log(np.log(sets_count[p2][1])), 1])
        mu_i = calculate_mu_i(X, sets_beta)
        
        sets_alpha = mu_i * sets_phi
        sets_beta = (1 - mu_i) * sets_phi
        k = (sets_count[p2][0] + sets_alpha) / (sets_count[p2][1] + sets_alpha + sets_beta)
    if p2 in tiebreaks_count and tiebreaks_count[p2][1] > 0:
        tiebreaks_beta = np.array([tiebreaks_beta0, tiebreaks_beta1])
        X = np.array([np.log(np.log(tiebreaks_count[p2][1])), 1])
        mu_i = calculate_mu_i(X, tiebreaks_beta)
        
        tiebreaks_alpha = mu_i * tiebreaks_phi
        tiebreaks_beta = (1 - mu_i) * tiebreaks_phi
        l = (tiebreaks_count[p2][0] + tiebreaks_alpha) / (tiebreaks_count[p2][1] + tiebreaks_alpha + tiebreaks_beta)
        
        
    m = serve_alpha
    n = serve_beta
    o = return_alpha
    p = return_beta
    q = service_game_alpha
    r = service_game_beta
    s = return_game_alpha
    t = return_game_beta
    u = sets_alpha
    v = sets_beta
    w = tiebreaks_alpha
    x = tiebreaks_beta
    
    
    return a,b,c,d,e,f,g,h,i,j,k,l,m,n,o,p,q,r,s,t,u,v,w,x





In [5]:
def compute_matchup_stats(match):
    s_itt = match["p1 tourney serve adv"]
    r_itt = match["p1 tourney return adv"]
    
    s_jtt = match["p2 tourney serve"]
    r_jtt = match["p2 tourney return adv"]
    
    sg_itt = match["p1 tourney service game adv"]
    rg_itt = match["p1 tourney return game adv"]
    
    sg_jtt = match["p2 tourney service game adv"]
    rg_jtt = match["p2 tourney return game adv"]
    
    sets_itt = match["p1 tourney set adv"]
    sets_jtt = match["p2 tourney set adv"]
    
    t_itt = match["p1 tourney tiebreak adv"]
    t_jtt = match["p2 tourney tiebreak adv"]
    
    # Pr[i wins serve on j at tournament t on day t]
    s_ijtt = (s_itt * (1 - r_jtt)) / ((s_itt * (1 - r_jtt)) + (r_jtt * (1 - s_itt)))
    # Pr[j wins serve on i at tournament t on day t]
    s_jitt = (s_jtt * (1 - r_itt)) / ((s_jtt * (1 - r_itt)) + (r_itt * (1 - s_jtt)))
    
    # Pr[i wins service game on j at tournament t on day t]
    sg_ijtt = (sg_itt * (1 - rg_jtt)) / ((sg_itt * (1 - rg_jtt)) + (rg_jtt * (1 - sg_itt)))
    # Pr[j wins service game on i at tournament t on day t]
    sg_jitt = (sg_jtt * (1 - rg_itt)) / ((sg_jtt * (1 - rg_itt)) + (rg_itt * (1 - sg_jtt)))
    
    # Pr[i wins set on j at tournament t on day t]
    sets_ijtt = (sets_itt * (1 - sets_jtt)) / ((sets_itt * (1 - sets_jtt)) + (sets_jtt * (1 - sets_itt)))
    
    # Pr[i wins tiebreak on j at tournament t on day t]
    t_ijtt = (t_itt * (1 - t_jtt)) / ((t_itt * (1 - t_jtt)) + (t_jtt * (1 - t_itt)))
    
    
    return s_ijtt, sg_ijtt, sets_ijtt, t_ijtt, s_jitt, sg_jitt

In [6]:
def compute_matchup_stats_beta(match):
    s_itt = match["p1 tourney serve adv beta"]
    r_itt = match["p1 tourney return adv beta"]
    
    s_jtt = match["p2 tourney serve adv beta"]
    r_jtt = match["p2 tourney return adv beta"]
    
    sg_itt = match["p1 tourney service game adv beta"]
    rg_itt = match["p1 tourney return game adv beta"]
    
    sg_jtt = match["p2 tourney service game adv beta"]
    rg_jtt = match["p2 tourney return game adv beta"]
    
    sets_itt = match["p1 tourney set adv beta"]
    sets_jtt = match["p2 tourney set adv beta"]
    
    t_itt = match["p1 tourney tiebreak adv beta"]
    t_jtt = match["p2 tourney tiebreak adv beta"]
    
    # Pr[i wins serve on j at tournament t on day t]
    s_ijtt = (s_itt * (1 - r_jtt)) / ((s_itt * (1 - r_jtt)) + (r_jtt * (1 - s_itt)))
    # Pr[j wins serve on i at tournament t on day t]
    s_jitt = (s_jtt * (1 - r_itt)) / ((s_jtt * (1 - r_itt)) + (r_itt * (1 - s_jtt)))
    
    # Pr[i wins service game on j at tournament t on day t]
    sg_ijtt = (sg_itt * (1 - rg_jtt)) / ((sg_itt * (1 - rg_jtt)) + (rg_jtt * (1 - sg_itt)))
    # Pr[j wins service game on i at tournament t on day t]
    sg_jitt = (sg_jtt * (1 - rg_itt)) / ((sg_jtt * (1 - rg_itt)) + (rg_itt * (1 - sg_jtt)))
    
    # Pr[i wins set on j at tournament t on day t]
    sets_ijtt = (sets_itt * (1 - sets_jtt)) / ((sets_itt * (1 - sets_jtt)) + (sets_jtt * (1 - sets_itt)))
    
    # Pr[i wins tiebreak on j at tournament t on day t]
    t_ijtt = (t_itt * (1 - t_jtt)) / ((t_itt * (1 - t_jtt)) + (t_jtt * (1 - t_itt)))
    
    
    return s_ijtt, sg_ijtt, sets_ijtt, t_ijtt, s_jitt, sg_jitt

In [7]:
def compute_matchup_stats_beta_reg(match):
    s_itt = match["p1 tourney serve adv beta reg"]
    r_itt = match["p1 tourney return adv beta reg"]
    
    s_jtt = match["p2 tourney serve adv beta reg"]
    r_jtt = match["p2 tourney return adv beta reg"]
    
    sg_itt = match["p1 tourney service game adv beta reg"]
    rg_itt = match["p1 tourney return game adv beta reg"]
    
    sg_jtt = match["p2 tourney service game adv beta reg"]
    rg_jtt = match["p2 tourney return game adv beta reg"]
    
    sets_itt = match["p1 tourney set adv beta reg"]
    sets_jtt = match["p2 tourney set adv beta reg"]
    
    t_itt = match["p1 tourney tiebreak adv beta reg"]
    t_jtt = match["p2 tourney tiebreak adv beta reg"]
    
    # Pr[i wins serve on j at tournament t on day t]
    s_ijtt = (s_itt * (1 - r_jtt)) / ((s_itt * (1 - r_jtt)) + (r_jtt * (1 - s_itt)))
    # Pr[j wins serve on i at tournament t on day t]
    s_jitt = (s_jtt * (1 - r_itt)) / ((s_jtt * (1 - r_itt)) + (r_itt * (1 - s_jtt)))
    
    # Pr[i wins service game on j at tournament t on day t]
    sg_ijtt = (sg_itt * (1 - rg_jtt)) / ((sg_itt * (1 - rg_jtt)) + (rg_jtt * (1 - sg_itt)))
    # Pr[j wins service game on i at tournament t on day t]
    sg_jitt = (sg_jtt * (1 - rg_itt)) / ((sg_jtt * (1 - rg_itt)) + (rg_itt * (1 - sg_jtt)))
    
    # Pr[i wins set on j at tournament t on day t]
    sets_ijtt = (sets_itt * (1 - sets_jtt)) / ((sets_itt * (1 - sets_jtt)) + (sets_jtt * (1 - sets_itt)))
    
    # Pr[i wins tiebreak on j at tournament t on day t]
    t_ijtt = (t_itt * (1 - t_jtt)) / ((t_itt * (1 - t_jtt)) + (t_jtt * (1 - t_itt)))
    
    
    return s_ijtt, sg_ijtt, sets_ijtt, t_ijtt, s_jitt, sg_jitt

# Beta Distribution Estimators

In [8]:
import pandas as pd
import numpy as np
from datetime import datetime
from dateutil.relativedelta import relativedelta
from scipy.stats import beta
pd.set_option("display.max_columns", None)

In [9]:
matches = pd.read_csv("./data/2011-2015_matches_cleaned_forShrinkage.csv")
matches["date"] = matches.apply(lambda x: datetime.strptime(x["date"], '%Y-%m-%d'), axis=1)
matches

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust
0,2231275,2011-07-28,ATPStu,ATP,Main,Olivier Rochus,Fabio Fognini,2,SSSS;RRRR;SSRRSS;SSRRSS;RSRSRSRR;SSRSS;RSRRSR;...,6-4 6-1,0,66,27,58,3,9,0,0,32,56,6,8,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2231276,2011-07-28,ATPStu,ATP,Main,Robin Haase,Marin Cilic,2,SSRSS;RRSSRSSS;SSSS;RSSSS;SRSRSS;RSRSRSSS;RSRS...,4-6 6-4 6-3,0,141,60,102,11,15,1,0,60,92,12,14,2,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2231342,2011-07-28,Farmer,ATP,Main,Alejandro Falla,Thomaz Bellucci,2,SRRSSRSRRSRSRSRSRSRR;SSSRS;RRSRR;SRSSS;RRSSRR;...,6-0 6-1,0,42,21,55,1,7,0,0,26,37,6,6,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,NaN,NaN,0,0,2,2,2,2,0,0,2,2,2,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2229262,2011-07-28,Credit,ATP,Main,Matthias Bachinger,Julien Benneteau,2,SSSS;SSSS;RRRR;SSSS;SSSS;SRRRSSRSSRRSSRSS;SSSS...,6-4 6-4,0,57,36,59,7,10,0,0,43,58,9,10,2,0,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [213]:
p1_beta_shrunken_cols =  ["p1 year serve beta", "p1 year return beta", "p1 year service game beta", "p1 year return game beta", "p1 year set beta", "p1 year tiebreak beta"]
p2_beta_shrunken_cols =  ["p2 year serve beta", "p2 year return beta", "p2 year service game beta", "p2 year return game beta", "p2 year set beta", "p2 year tiebreak beta"]
beta_shrunken_cols = p1_beta_shrunken_cols + p2_beta_shrunken_cols
greek_cols = ["serve alpha", "serve beta", "return alpha", "return beta", "service game alpha", "service game beta", "return game alpha", "return game beta", "set alpha", "set beta", "tiebreak alpha", "tiebreak beta"]
beta_shrunken_cols += greek_cols
match_count_threshold = 4

testing = matches.tail(100)

testing[beta_shrunken_cols] = testing.apply(lambda match: calculate_beta_binomial_regression_estimators_past_year(match, matches, match_count_threshold), axis=1, result_type="expand")



testing


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log


KeyError: 'serve alpha'

In [10]:
p1_beta_shrunken_cols =  ["p1 year serve beta", "p1 year return beta", "p1 year service game beta", "p1 year return game beta", "p1 year set beta", "p1 year tiebreak beta"]
p2_beta_shrunken_cols =  ["p2 year serve beta", "p2 year return beta", "p2 year service game beta", "p2 year return game beta", "p2 year set beta", "p2 year tiebreak beta"]
beta_shrunken_cols = p1_beta_shrunken_cols + p2_beta_shrunken_cols
greek_cols = ["serve alpha", "serve beta", "return alpha", "return beta", "service game alpha", "service game beta", "return game alpha", "return game beta", "set alpha", "set beta", "tiebreak alpha", "tiebreak beta"]
beta_shrunken_cols += greek_cols
match_count_threshold = 2

matches[beta_shrunken_cols] = matches.apply(lambda match: calculate_beta_binomial_estimators_past_year(match, matches, match_count_threshold), axis=1, result_type="expand")



In [11]:
# calculate no sos adjust p1 serve_tt, pl_service_game_tt, etc.
matches["p1 serve_tt beta no sos adjust"] = (matches["p1 year serve beta"] * (1 - matches["p2 year return beta"])) / ((matches["p1 year serve beta"] * (1 - matches["p2 year return beta"])) + (matches["p2 year return beta"] * (1 - matches["p1 year serve beta"])))
matches["p2 serve_tt beta no sos adjust"] = (matches["p2 year serve beta"] * (1 - matches["p1 year return beta"])) / ((matches["p2 year serve beta"] * (1 - matches["p1 year return beta"])) + (matches["p1 year return beta"] * (1 - matches["p2 year serve beta"])))

matches["p1 service_game_tt beta no sos adjust"] = (matches["p1 year service game beta"] * (1 - matches["p2 year return game beta"])) / ((matches["p1 year service game beta"] * (1 - matches["p2 year return game beta"])) + (matches["p2 year return game beta"] * (1 - matches["p1 year service game beta"])))
matches["p2 service_game_tt beta no sos adjust"] = (matches["p2 year service game beta"] * (1 - matches["p1 year return game beta"])) / ((matches["p2 year service game beta"] * (1 - matches["p1 year return game beta"])) + (matches["p1 year return game beta"] * (1 - matches["p2 year service game beta"])))

matches["p1 tiebreak_tt beta no sos adjust"] = (matches["p1 year tiebreak beta"] * (1 - matches["p2 year tiebreak beta"])) / ((matches["p1 year tiebreak beta"] * (1 - matches["p2 year tiebreak beta"])) + (matches["p2 year tiebreak beta"] * (1 - matches["p1 year tiebreak beta"])))

matches["p1 set_tt beta no sos adjust"] = (matches["p1 year set beta"] * (1 - matches["p2 year set beta"])) / ((matches["p1 year set beta"] * (1 - matches["p2 year set beta"])) + (matches["p2 year set beta"] * (1 - matches["p1 year set beta"])))




In [152]:
p1_beta_reg_shrunken_cols =  ["p1 year serve beta reg", "p1 year return beta reg", "p1 year service game beta reg", "p1 year return game beta reg", "p1 year set beta reg", "p1 year tiebreak beta reg"]
p2_beta_reg_shrunken_cols =  ["p2 year serve beta reg", "p2 year return beta reg", "p2 year service game beta reg", "p2 year return game beta reg", "p2 year set beta reg", "p2 year tiebreak beta reg"]
beta_reg_shrunken_cols = p1_beta_reg_shrunken_cols + p2_beta_reg_shrunken_cols
greek_cols = ["serve alpha reg p2", "serve beta reg p2", "return alpha reg p2", "return beta reg p2", "service game alpha reg p2", "service game beta reg p2", "return game alpha reg p2", "return game beta reg p2", "set alpha reg p2", "set beta reg p2", "tiebreak alpha reg p2", "tiebreak beta reg p2"]
beta_reg_shrunken_cols += greek_cols
match_count_threshold = 2

matches[beta_reg_shrunken_cols] = matches.apply(lambda match: calculate_beta_binomial_regression_estimators_past_year(match, matches, match_count_threshold), axis=1, result_type="expand")

/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:148: RuntimeWarning: divide by zero encountered in log
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:198: RuntimeWarning: divide by zero encountered in log
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:41: RuntimeWarning: divide by zero encountered in log


In [153]:
# calculate no sos adjust p1 serve_tt, pl_service_game_tt, etc.
matches["p1 serve_tt beta reg no sos adjust"] = (matches["p1 year serve beta reg"] * (1 - matches["p2 year return beta reg"])) / ((matches["p1 year serve beta reg"] * (1 - matches["p2 year return beta reg"])) + (matches["p2 year return beta reg"] * (1 - matches["p1 year serve beta reg"])))
matches["p2 serve_tt beta reg no sos adjust"] = (matches["p2 year serve beta reg"] * (1 - matches["p1 year return beta reg"])) / ((matches["p2 year serve beta reg"] * (1 - matches["p1 year return beta reg"])) + (matches["p1 year return beta reg"] * (1 - matches["p2 year serve beta reg"])))

matches["p1 service_game_tt beta reg no sos adjust"] = (matches["p1 year service game beta reg"] * (1 - matches["p2 year return game beta reg"])) / ((matches["p1 year service game beta reg"] * (1 - matches["p2 year return game beta reg"])) + (matches["p2 year return game beta reg"] * (1 - matches["p1 year service game beta reg"])))
matches["p2 service_game_tt beta reg no sos adjust"] = (matches["p2 year service game beta reg"] * (1 - matches["p1 year return game beta reg"])) / ((matches["p2 year service game beta reg"] * (1 - matches["p1 year return game beta reg"])) + (matches["p1 year return game beta reg"] * (1 - matches["p2 year service game beta reg"])))

matches["p1 tiebreak_tt beta reg no sos adjust"] = (matches["p1 year tiebreak beta reg"] * (1 - matches["p2 year tiebreak beta reg"])) / ((matches["p1 year tiebreak beta reg"] * (1 - matches["p2 year tiebreak beta reg"])) + (matches["p2 year tiebreak beta reg"] * (1 - matches["p1 year tiebreak beta reg"])))

matches["p1 set_tt beta reg no sos adjust"] = (matches["p1 year set beta reg"] * (1 - matches["p2 year set beta reg"])) / ((matches["p1 year set beta reg"] * (1 - matches["p2 year set beta reg"])) + (matches["p2 year set beta reg"] * (1 - matches["p1 year set beta reg"])))




In [154]:
matches

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust,p1 year serve beta,p1 year return beta,p1 year service game beta,p1 year return game beta,p1 year set beta,p1 year tiebreak beta,p2 year serve beta,p2 year return beta,p2 year service game beta,p2 year return game beta,p2 year set beta,p2 year tiebreak beta,serve alpha,serve beta,return alpha,return beta,service game alpha,service game beta,return game alpha,return game beta,set alpha,set beta,tiebreak alpha,tiebreak beta,p1 serve_tt beta no sos adjust,p2 serve_tt beta no sos adjust,p1 service_game_tt beta no sos adjust,p2 service_game_tt beta no sos adjust,p1 tiebreak_tt beta no sos adjust,p1 set_tt beta no sos adjust,p1 year serve beta reg,p1 year return beta reg,p1 year service game beta reg,p1 year return game beta reg,p1 year set beta reg,p1 year tiebreak beta reg,p2 year serve beta reg,p2 year return beta reg,p2 year service game beta reg,p2 year return game beta reg,p2 year set beta reg,p2 year tiebreak beta reg,serve alpha reg p2,serve beta reg p2,return alpha reg p2,return beta reg p2,service game alpha reg p2,service game beta reg p2,return game alpha reg p2,return game beta reg p2,set alpha reg p2,set beta reg p2,tiebreak alpha reg p2,tiebreak beta reg p2,p1 serve_tt beta reg no sos adjust,p2 serve_tt beta reg no sos adjust,p1 service_game_tt beta reg no sos adjust,p2 service_game_tt beta reg no sos adjust,p1 tiebreak_tt beta reg no sos adjust,p1 set_tt beta reg no sos adjust,p1 relative serve beta,p2 relative serve beta,p1 relative return beta,p2 relative return beta,p1 relative service games beta,p2 relative service games beta,p1 relative return games beta,p2 relative return games beta,p1 relative set beta,p2 relative set beta,p1 relative tiebreak beta,p2 relat

In [155]:
# Generate relative abilities beta, we use unshrunken opp stats (allows a possibility for improvement)
matches["p1 relative serve beta"] = matches["p1 year serve beta"] - (1 - matches["p1 opp return win rate"])
matches["p2 relative serve beta"] = matches["p2 year serve beta"] - (1 - matches["p2 opp return win rate"])

matches["p1 relative return beta"] = matches["p1 year return beta"] - (1 - matches["p1 opp serve win rate"])
matches["p2 relative return beta"] = matches["p2 year return beta"] - (1 - matches["p2 opp serve win rate"])

matches["p1 relative service games beta"] = matches["p1 year service game beta"] - (1 - matches["p1 opp return game win rate"])
matches["p2 relative service games beta"] = matches["p2 year service game beta"] - (1 - matches["p2 opp return game win rate"])

matches["p1 relative return games beta"] = matches["p1 year return game beta"] - (1 - matches["p1 opp service game win rate"])
matches["p2 relative return games beta"] = matches["p2 year return game beta"] - (1 - matches["p2 opp service game win rate"])

matches["p1 relative set beta"] = matches["p1 year set beta"] - (1 - matches["p1 opp set win rate"])
matches["p2 relative set beta"] = matches["p2 year set beta"] - (1 - matches["p2 opp set win rate"])

matches["p1 relative tiebreak beta"] = matches["p1 year tiebreak beta"] - (1 - matches["p1 opp tiebreak win rate"])
matches["p2 relative tiebreak beta"] = matches["p2 year tiebreak beta"] - (1 - matches["p2 opp tiebreak win rate"])

In [156]:
# Generate relative abilities beta reg, we use unshrunken opp stats (allows a possibility for improvement)
matches["p1 relative serve beta reg"] = matches["p1 year serve beta reg"] - (1 - matches["p1 opp return win rate"])
matches["p2 relative serve beta reg"] = matches["p2 year serve beta reg"] - (1 - matches["p2 opp return win rate"])

matches["p1 relative return beta reg"] = matches["p1 year return beta reg"] - (1 - matches["p1 opp serve win rate"])
matches["p2 relative return beta reg"] = matches["p2 year return beta reg"] - (1 - matches["p2 opp serve win rate"])

matches["p1 relative service games beta reg"] = matches["p1 year service game beta reg"] - (1 - matches["p1 opp return game win rate"])
matches["p2 relative service games beta reg"] = matches["p2 year service game beta reg"] - (1 - matches["p2 opp return game win rate"])

matches["p1 relative return games beta reg"] = matches["p1 year return game beta reg"] - (1 - matches["p1 opp service game win rate"])
matches["p2 relative return games beta reg"] = matches["p2 year return game beta reg"] - (1 - matches["p2 opp service game win rate"])

matches["p1 relative set beta reg"] = matches["p1 year set beta reg"] - (1 - matches["p1 opp set win rate"])
matches["p2 relative set beta reg"] = matches["p2 year set beta reg"] - (1 - matches["p2 opp set win rate"])

matches["p1 relative tiebreak beta reg"] = matches["p1 year tiebreak beta reg"] - (1 - matches["p1 opp tiebreak win rate"])
matches["p2 relative tiebreak beta reg"] = matches["p2 year tiebreak beta reg"] - (1 - matches["p2 opp tiebreak win rate"])

In [157]:
# Calculate tournament averages
matches["tourney serve win rate"] = matches["tourney serves won"] / matches["tourney serves total"]
matches["tourney return win rate"] = 1 - matches["tourney serve win rate"]
matches["tourney service game win rate"] = matches["tourney service games won"] / matches["tourney service games total"]
matches["tourney return game win rate"] = 1 - matches["tourney service game win rate"]

In [158]:
# Computing player-tournament stats for beta shrunken vals
matches["p1 tourney serve adv beta"] = (matches["tourney serve win rate"] + matches["p1 relative serve beta"]).clip(upper=0.9985, lower=0.0015)
matches["p1 tourney return adv beta"] = (matches["tourney return win rate"] + matches["p1 relative return beta"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney serve adv beta"] = (matches["tourney serve win rate"] + matches["p2 relative serve beta"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney return adv beta"] = (matches["tourney return win rate"] + matches["p2 relative return beta"]).clip(upper=0.9985, lower=0.0015)

matches["p1 tourney service game adv beta"] = (matches["tourney service game win rate"] + matches["p1 relative service games beta"]).clip(upper=0.9985, lower=0.0015)
matches["p1 tourney return game adv beta"] = (matches["tourney return game win rate"] + matches["p1 relative return games beta"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney service game adv beta"] = (matches["tourney service game win rate"] + matches["p2 relative service games beta"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney return game adv beta"] = (matches["tourney return game win rate"] + matches["p2 relative return games beta"]).clip(upper=0.9985, lower=0.0015)

matches["p1 tourney set adv beta"] = (0.5 + matches["p1 relative set beta"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney set adv beta"] = (0.5 + matches["p2 relative set beta"]).clip(upper=0.9985, lower=0.0015)

matches["p1 tourney tiebreak adv beta"] = (0.5 + matches["p1 relative tiebreak beta"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney tiebreak adv beta"] = (0.5 + matches["p2 relative tiebreak beta"]).clip(upper=0.9985, lower=0.0015)

In [159]:
# Computing player-tournament stats for beta regression shrunken values
matches["p1 tourney serve adv beta reg"] = (matches["tourney serve win rate"] + matches["p1 relative serve beta reg"]).clip(upper=0.9985, lower=0.0015)
matches["p1 tourney return adv beta reg"] = (matches["tourney return win rate"] + matches["p1 relative return beta reg"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney serve adv beta reg"] = (matches["tourney serve win rate"] + matches["p2 relative serve beta reg"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney return adv beta reg"] = (matches["tourney return win rate"] + matches["p2 relative return beta reg"]).clip(upper=0.9985, lower=0.0015)

matches["p1 tourney service game adv beta reg"] = (matches["tourney service game win rate"] + matches["p1 relative service games beta reg"]).clip(upper=0.9985, lower=0.0015)
matches["p1 tourney return game adv beta reg"] = (matches["tourney return game win rate"] + matches["p1 relative return games beta reg"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney service game adv beta reg"] = (matches["tourney service game win rate"] + matches["p2 relative service games beta reg"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney return game adv beta reg"] = (matches["tourney return game win rate"] + matches["p2 relative return games beta reg"]).clip(upper=0.9985, lower=0.0015)

matches["p1 tourney set adv beta reg"] = (0.5 + matches["p1 relative set beta reg"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney set adv beta reg"] = (0.5 + matches["p2 relative set beta reg"]).clip(upper=0.9985, lower=0.0015)

matches["p1 tourney tiebreak adv beta reg"] = (0.5 + matches["p1 relative tiebreak beta reg"]).clip(upper=0.9985, lower=0.0015)
matches["p2 tourney tiebreak adv beta reg"] = (0.5 + matches["p2 relative tiebreak beta reg"]).clip(upper=0.9985, lower=0.0015)

# Start computation of logloss using Beta values (no opp/ tourney adjustment)

In [160]:
# compute sets needed to win match (to use later in row filtering for evaluation)
# Computing sets needed to win the match
matches["tourney sets to win"] = matches[["p1 sets won", "p2 sets won"]].max(axis=1)

In [161]:
# Computing matchup statistics
matchup_cols = ["p1 serve_tt", "p1 service_game_tt", "p1 set_tt", "p1 tiebreak_tt", "p2 serve_tt", "p2 service_game_tt"]
matches[matchup_cols] = matches.apply(lambda match: compute_matchup_stats_beta(match), axis=1, result_type="expand")

### Predict using $s_{ijtt}$ and $r_{ijtt}$

In [162]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [163]:
# Remove rows where we don't have s_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 serve_tt'].notna()) & (games_after_2012['p2 serve_tt'].notna())]

In [164]:
# import the markov model functions lazily
%run Modeling_tennis_match_iid_points.ipynb

0.70107
0.933922


In [165]:
# compute Pr[p1 wins] with opponent/ tournament adjusted shrunken serve values
games_after_2012["p1 pr win"] = games_after_2012.apply(lambda match: match_driver(2, 6, 4, 7, 1, match["p1 serve_tt"], match["p2 serve_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)



In [166]:
# compute Pr[p1 wins] with non sos adjusted JS serve rates
games_after_2012["p1 pr win beta no sos adjust"] = games_after_2012.apply(lambda match: match_driver(2, 6, 4, 7, 1, match["p1 serve_tt beta no sos adjust"], match["p2 serve_tt beta no sos adjust"]), axis=1).clip(upper=0.9985, lower=0.0015)




In [167]:
# compute log loss
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win", "p1 pr win beta no sos adjust"]]

compute_lloss["lloss sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win"]) + (1-m["winner"])*np.log(1 - m["p1 pr win"])), axis=1)
compute_lloss["lloss no sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win beta no sos adjust"]) + (1-m["winner"])*np.log(1 - m["p1 pr win beta no sos adjust"])), axis=1)
N = compute_lloss.shape[0]

lloss_sos_adjust = (compute_lloss["lloss sos adjust"].sum()) * (-1/N)
lloss_no_sos_adjust = (compute_lloss["lloss no sos adjust"].sum()) * (-1/N)

print(f"Sos adjust log loss: {lloss_sos_adjust}")
print(f"No sos adjust log loss: {lloss_no_sos_adjust}")
print(f"Random guessing log loss: {-np.log(0.5)}")

Sos adjust log loss: 0.6157244175045516
No sos adjust log loss: 0.613987901327833
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### Predict using $sg_{ijtt}$ and $rg_{ijtt}$ and $t_{ijtt}$

In [168]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [169]:
# Remove rows where we don't have sg_ijtt and t_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 service_game_tt'].notna()) & (games_after_2012['p2 service_game_tt'].notna()) & (games_after_2012['p1 tiebreak_tt'].notna())]



In [170]:

# compute Pr[p1 wins] with non opponenent/ tournament adjusted service game values
games_after_2012["p1 pr win opp adjust game based"] = games_after_2012.apply(lambda match: match_driver_game_based(2, 6, 4, 7, 1, match["p1 service_game_tt"], match["p2 service_game_tt"], match["p1 tiebreak_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
games_after_2012["p1 pr win beta no sos adjust game based"] = games_after_2012.apply(lambda match: match_driver_game_based(2, 6, 4, 7, 1, match["p1 service_game_tt beta no sos adjust"], match["p2 service_game_tt beta no sos adjust"], match["p1 tiebreak_tt beta no sos adjust"]), axis=1).clip(upper=0.9985, lower=0.0015)




In [171]:
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win opp adjust game based", "p1 pr win beta no sos adjust game based"]]

compute_lloss["lloss sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win opp adjust game based"]) + (1-m["winner"])*np.log(1 - m["p1 pr win opp adjust game based"])), axis=1)
compute_lloss["lloss beta no sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win beta no sos adjust game based"]) + (1-m["winner"])*np.log(1 - m["p1 pr win beta no sos adjust game based"])), axis=1)
N = compute_lloss.shape[0]

lloss_sos_adjust = (compute_lloss["lloss sos adjust"].sum()) * (-1/N)
lloss_no_sos_adjust = (compute_lloss["lloss beta no sos adjust"].sum()) * (-1/N)

print(f"Sos adjust log loss: {lloss_sos_adjust}")
print(f"No sos adjust log loss: {lloss_no_sos_adjust}")
print(f"Random guessing log loss: {-np.log(0.5)}")

Sos adjust log loss: 0.6192710019643539
No sos adjust log loss: 0.6278290268579049
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Try to predict games using $sets_{ijtt}$ values

In [172]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [173]:
# Remove rows where we don't have sg_ijtt and t_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 set_tt'].notna())]

In [174]:

# compute Pr[p1 wins] with non opponenent/ tournament adjusted service game values
games_after_2012["p1 pr win opp adjust set based"] = games_after_2012.apply(lambda match: match_driver_set_based(2, match["p1 set_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
games_after_2012["p1 pr win beta no sos adjust set based"] = games_after_2012.apply(lambda match: match_driver_set_based(2, match["p1 set_tt beta no sos adjust"]), axis=1).clip(upper=0.9985, lower=0.0015)




In [175]:
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win opp adjust set based", "p1 pr win beta no sos adjust set based"]]

compute_lloss["lloss sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win opp adjust set based"]) + (1-m["winner"])*np.log(1 - m["p1 pr win opp adjust set based"])), axis=1)
compute_lloss["lloss no sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win beta no sos adjust set based"]) + (1-m["winner"])*np.log(1 - m["p1 pr win beta no sos adjust set based"])), axis=1)
N = compute_lloss.shape[0]

lloss_sos_adjust = (compute_lloss["lloss sos adjust"].sum()) * (-1/N)
lloss_no_sos_adjust = (compute_lloss["lloss no sos adjust"].sum()) * (-1/N)

print(f"Sos adjust log loss: {lloss_sos_adjust}")
print(f"No sos adjust log loss: {lloss_no_sos_adjust}")
print(f"Random guessing log loss: {-np.log(0.5)}")

/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


Sos adjust log loss: 0.6047965975325152
No sos adjust log loss: 0.6089196475903351
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Try to predict using $sg_{ijtt}$ for games and $s_{ijtt}, s_{jitt}$ for tiebreaks

In [176]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [177]:
# Remove rows where we don't have sg_ijtt and s_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 service_game_tt'].notna()) & (games_after_2012['p2 service_game_tt'].notna()) & (games_after_2012['p1 serve_tt'].notna()) & (games_after_2012['p2 serve_tt'].notna())]

In [178]:
# compute Pr[p1 wins] with non opponenent/ tournament adjusted service game values
games_after_2012["p1 pr win opp adjust mix"] = games_after_2012.apply(lambda match: match_driver_game_based_tiebreak_serve_based(2, 6, 4, 7, 1, match["p1 service_game_tt"], match["p2 service_game_tt"], match["p1 serve_tt"], match["p2 serve_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
games_after_2012["p1 pr win beta no sos adjust mix"] = games_after_2012.apply(lambda match: match_driver_game_based_tiebreak_serve_based(2, 6, 4, 7, 1, match["p1 service_game_tt beta no sos adjust"], match["p2 service_game_tt beta no sos adjust"], match["p1 serve_tt beta no sos adjust"], match["p2 serve_tt beta no sos adjust"]), axis=1).clip(upper=0.9985, lower=0.0015)






In [179]:
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win opp adjust mix", "p1 pr win beta no sos adjust mix"]]

compute_lloss["lloss sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win opp adjust mix"]) + (1-m["winner"])*np.log(1 - m["p1 pr win opp adjust mix"])), axis=1)
compute_lloss["lloss no sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win beta no sos adjust mix"]) + (1-m["winner"])*np.log(1 - m["p1 pr win beta no sos adjust mix"])), axis=1)
N = compute_lloss.shape[0]

lloss_sos_adjust = (compute_lloss["lloss sos adjust"].sum()) * (-1/N)
lloss_no_sos_adjust = (compute_lloss["lloss no sos adjust"].sum()) * (-1/N)

print(f"Sos adjust log loss: {lloss_sos_adjust}")
print(f"No sos adjust log loss: {lloss_no_sos_adjust}")
print(f"Random guessing log loss: {-np.log(0.5)}")

Sos adjust log loss: 0.6034121538496443
No sos adjust log loss: 0.6133881871571246
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


# Start computation of logloss using Beta reg values (no opp/ tourney adjustment)

In [180]:
# Computing matchup statistics
matchup_cols = ["p1 serve_tt", "p1 service_game_tt", "p1 set_tt", "p1 tiebreak_tt", "p2 serve_tt", "p2 service_game_tt"]
matches[matchup_cols] = matches.apply(lambda match: compute_matchup_stats_beta_reg(match), axis=1, result_type="expand")

### Predict using $s_{ijtt}$ and $r_{ijtt}$

In [181]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [182]:
# Remove rows where we don't have s_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 serve_tt'].notna()) & (games_after_2012['p2 serve_tt'].notna())]

In [183]:
# import the markov model functions lazily
%run Modeling_tennis_match_iid_points.ipynb

0.6979
0.933309


In [184]:
# compute Pr[p1 wins] with opponent/ tournament adjusted shrunken serve values
games_after_2012["p1 pr win"] = games_after_2012.apply(lambda match: match_driver(2, 6, 4, 7, 1, match["p1 serve_tt"], match["p2 serve_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)



In [185]:
# compute Pr[p1 wins] with non sos adjusted JS serve rates
games_after_2012["p1 pr win beta reg no sos adjust"] = games_after_2012.apply(lambda match: match_driver(2, 6, 4, 7, 1, match["p1 serve_tt beta reg no sos adjust"], match["p2 serve_tt beta reg no sos adjust"]), axis=1).clip(upper=0.9985, lower=0.0015)




In [186]:
# compute log loss
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win", "p1 pr win beta reg no sos adjust"]]

compute_lloss["lloss sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win"]) + (1-m["winner"])*np.log(1 - m["p1 pr win"])), axis=1)
compute_lloss["lloss no sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win beta reg no sos adjust"]) + (1-m["winner"])*np.log(1 - m["p1 pr win beta reg no sos adjust"])), axis=1)
N = compute_lloss.shape[0]

lloss_sos_adjust = (compute_lloss["lloss sos adjust"].sum()) * (-1/N)
lloss_no_sos_adjust = (compute_lloss["lloss no sos adjust"].sum()) * (-1/N)

print(f"Sos adjust log loss: {lloss_sos_adjust}")
print(f"No sos adjust log loss: {lloss_no_sos_adjust}")
print(f"Random guessing log loss: {-np.log(0.5)}")

Sos adjust log loss: 0.6138331800869373
No sos adjust log loss: 0.611819436703535
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


### Predict using $sg_{ijtt}$ and $rg_{ijtt}$ and $t_{ijtt}$

In [195]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [196]:
# Remove rows where we don't have sg_ijtt and t_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 service_game_tt'].notna()) & (games_after_2012['p2 service_game_tt'].notna()) & (games_after_2012['p1 tiebreak_tt'].notna())]



In [197]:
games_after_2012.apply(lambda x: x)

,pbp_id,date,tny_name,tour,draw,server1,server2,winner,pbp,score,adf_flag,wh_minutes,p1 serves won,p1 serves total,p1 service games won,p1 service games total,p1 sets won,p1 tiebreaks won,p2 serves won,p2 serves total,p2 service games won,p2 service games total,p2 sets won,p2 tiebreaks won,sets total,tourney serves won,tourney serves total,tourney service games won,tourney service games total,p1 serve win rate,p1 return win rate,p1 service game win rate,p1 return game win rate,p1 set win rate,p1 tiebreak win rate,p2 serve win rate,p2 return win rate,p2 service game win rate,p2 return game win rate,p2 set win rate,p2 tiebreak win rate,p1 opp serve win rate,p1 opp return win rate,p1 opp service game win rate,p1 opp return game win rate,p1 opp set win rate,p1 opp tiebreak win rate,p2 opp serve win rate,p2 opp return win rate,p2 opp service game win rate,p2 opp return game win rate,p2 opp set win rate,p2 opp tiebreak win rate,p1 match count,p2 match count,all player year serve win rate,all player year service game win rate,p1 year serve total,p1 year return total,p1 year service game total,p1 year return game total,p1 year set total,p1 year tiebreak total,p2 year serve total,p2 year return total,p2 year service game total,p2 year return game total,p2 year set total,p2 year tiebreak total,p1 year match win rate,p2 year match rate,p1 year match results,p2 year match results,p1 opp match win rate,p2 opp match win rate,p1 relative serve,p2 relative serve,p1 relative return,p2 relative return,p1 relative service games,p2 relative service games,p1 relative return games,p2 relative return games,p1 relative set,p2 relative set,p1 relative tiebreak,p2 relative tiebreak,tourney serve win rate,tourney return win rate,tourney service game win rate,tourney return game win rate,p1 tourney serve adv,p1 tourney return adv,p2 tourney serve adv,p2 tourney return adv,p1 tourney service game adv,p1 tourney return game adv,p2 tourney service game adv,p2 tourney return game adv,p1 tourney set adv,p2 tourney set adv,p1 tourney tiebreak adv,p2 tourney tiebreak adv,tourney sets to win,p1 serve_tt,p1 service_game_tt,p1 set_tt,p1 tiebreak_tt,p2 serve_tt,p2 service_game_tt,p1 serve_tt no sos adjust,p2 serve_tt no sos adjust,p1 service_game_tt no sos adjust,p2 service_game_tt no sos adjust,p1 year serve beta,p1 year return beta,p1 year service game beta,p1 year return game beta,p1 year set beta,p1 year tiebreak beta,p2 year serve beta,p2 year return beta,p2 year service game beta,p2 year return game beta,p2 year set beta,p2 year tiebreak beta,serve alpha,serve beta,return alpha,return beta,service game alpha,service game beta,return game alpha,return game beta,set alpha,set beta,tiebreak alpha,tiebreak beta,p1 serve_tt beta no sos adjust,p2 serve_tt beta no sos adjust,p1 service_game_tt beta no sos adjust,p2 service_game_tt beta no sos adjust,p1 tiebreak_tt beta no sos adjust,p1 set_tt beta no sos adjust,p1 year serve beta reg,p1 year return beta reg,p1 year service game beta reg,p1 year return game beta reg,p1 year set beta reg,p1 year tiebreak beta reg,p2 year serve beta reg,p2 year return beta reg,p2 year service game beta reg,p2 year return game beta reg,p2 year set beta reg,p2 year tiebreak beta reg,serve alpha reg p2,serve beta reg p2,return alpha reg p2,return beta reg p2,service game alpha reg p2,service game beta reg p2,return game alpha reg p2,return game beta reg p2,set alpha reg p2,set beta reg p2,tiebreak alpha reg p2,tiebreak beta reg p2,p1 serve_tt beta reg no sos adjust,p2 serve_tt beta reg no sos adjust,p1 service_game_tt beta reg no sos adjust,p2 service_game_tt beta reg no sos adjust,p1 tiebreak_tt beta reg no sos adjust,p1 set_tt beta reg no sos adjust,p1 relative serve beta,p2 relative serve beta,p1 relative return beta,p2 relative return beta,p1 relative service games beta,p2 relative service games beta,p1 relative return games beta,p2 relative return games beta,p1 relative set beta,p2 relative set beta,p1 relative tiebreak beta,p2 relat

In [198]:

# compute Pr[p1 wins] with non opponenent/ tournament adjusted service game values
games_after_2012["p1 pr win opp adjust game based"] = games_after_2012.apply(lambda match: match_driver_game_based(2, 6, 4, 7, 1, match["p1 service_game_tt"], match["p2 service_game_tt"], match["p1 tiebreak_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
games_after_2012["p1 pr win beta reg no sos adjust game based"] = games_after_2012.apply(lambda match: match_driver_game_based(2, 6, 4, 7, 1, match["p1 service_game_tt beta reg no sos adjust"], match["p2 service_game_tt beta reg no sos adjust"], match["p1 tiebreak_tt beta reg no sos adjust"]), axis=1).clip(upper=0.9985, lower=0.0015)




ValueError: Wrong number of items passed 222, placement implies 1

In [ ]:
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win opp adjust game based", "p1 pr win beta reg no sos adjust game based"]]

compute_lloss["lloss sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win opp adjust game based"]) + (1-m["winner"])*np.log(1 - m["p1 pr win opp adjust game based"])), axis=1)
compute_lloss["lloss beta reg no sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win beta reg no sos adjust game based"]) + (1-m["winner"])*np.log(1 - m["p1 pr win beta reg no sos adjust game based"])), axis=1)
N = compute_lloss.shape[0]

lloss_sos_adjust = (compute_lloss["lloss sos adjust"].sum()) * (-1/N)
lloss_no_sos_adjust = (compute_lloss["lloss beta reg no sos adjust"].sum()) * (-1/N)

print(f"Sos adjust log loss: {lloss_sos_adjust}")
print(f"No sos adjust log loss: {lloss_no_sos_adjust}")
print(f"Random guessing log loss: {-np.log(0.5)}")

In [ ]:
matches[["p1 service_game_tt beta reg no sos adjust","p2 service_game_tt beta reg no sos adjust", "p1 tiebreak_tt beta reg no sos adjust","p1 tourney tiebreak adv beta reg","p2 tourney tiebreak adv beta reg"]].describe()





### Try to predict games using $sets_{ijtt}$ values

In [187]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [188]:
# Remove rows where we don't have sg_ijtt and t_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 set_tt'].notna())]

In [189]:

# compute Pr[p1 wins] with non opponenent/ tournament adjusted service game values
games_after_2012["p1 pr win opp adjust set based"] = games_after_2012.apply(lambda match: match_driver_set_based(2, match["p1 set_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
games_after_2012["p1 pr win beta reg no sos adjust set based"] = games_after_2012.apply(lambda match: match_driver_set_based(2, match["p1 set_tt beta reg no sos adjust"]), axis=1).clip(upper=0.9985, lower=0.0015)




In [190]:
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win opp adjust set based", "p1 pr win beta reg no sos adjust set based"]]

compute_lloss["lloss sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win opp adjust set based"]) + (1-m["winner"])*np.log(1 - m["p1 pr win opp adjust set based"])), axis=1)
compute_lloss["lloss no sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win beta reg no sos adjust set based"]) + (1-m["winner"])*np.log(1 - m["p1 pr win beta reg no sos adjust set based"])), axis=1)
N = compute_lloss.shape[0]

lloss_sos_adjust = (compute_lloss["lloss sos adjust"].sum()) * (-1/N)
lloss_no_sos_adjust = (compute_lloss["lloss no sos adjust"].sum()) * (-1/N)

print(f"Sos adjust log loss: {lloss_sos_adjust}")
print(f"No sos adjust log loss: {lloss_no_sos_adjust}")
print(f"Random guessing log loss: {-np.log(0.5)}")

Sos adjust log loss: 0.6291160115060419
No sos adjust log loss: 0.6100998412663834
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


### Try to predict using $sg_{ijtt}$ for games and $s_{ijtt}, s_{jitt}$ for tiebreaks

In [191]:
# Filter matches we want to predict on

# Only consider matches on or after August 1, 2012, because we've had one year of data at that point
games_after_2012 = matches[matches["date"] > datetime.strptime("31 Dec 14", '%d %b %y')]

# Only consider best of three set matches
games_after_2012 = games_after_2012[games_after_2012["tourney sets to win"] == 2]

# Don't consider Davis Cup matches because they don't have a final set tiebreak
games_after_2012 = games_after_2012[games_after_2012["tny_name"] != "DavisC"]

# Remove rows where either player hasn't played at least 10 matches in the past year
games_after_2012 = games_after_2012[(games_after_2012["p1 match count"] >= 2) & (games_after_2012["p2 match count"] >= 2)]

In [192]:
# Remove rows where we don't have sg_ijtt and s_ijtt values to predict the result of the match
games_after_2012 = games_after_2012[(games_after_2012['p1 service_game_tt'].notna()) & (games_after_2012['p2 service_game_tt'].notna()) & (games_after_2012['p1 serve_tt'].notna()) & (games_after_2012['p2 serve_tt'].notna())]

In [193]:
# compute Pr[p1 wins] with non opponenent/ tournament adjusted service game values
games_after_2012["p1 pr win opp adjust mix"] = games_after_2012.apply(lambda match: match_driver_game_based_tiebreak_serve_based(2, 6, 4, 7, 1, match["p1 service_game_tt"], match["p2 service_game_tt"], match["p1 serve_tt"], match["p2 serve_tt"]), axis=1).clip(upper=0.9985, lower=0.0015)
games_after_2012["p1 pr win beta reg no sos adjust mix"] = games_after_2012.apply(lambda match: match_driver_game_based_tiebreak_serve_based(2, 6, 4, 7, 1, match["p1 service_game_tt beta reg no sos adjust"], match["p2 service_game_tt beta reg no sos adjust"], match["p1 serve_tt beta reg no sos adjust"], match["p2 serve_tt beta reg no sos adjust"]), axis=1).clip(upper=0.9985, lower=0.0015)






In [194]:
games_after_2012["winner"] = games_after_2012["winner"].replace(2,0)

compute_lloss = games_after_2012[["winner", "p1 pr win opp adjust mix", "p1 pr win beta reg no sos adjust mix"]]

compute_lloss["lloss sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win opp adjust mix"]) + (1-m["winner"])*np.log(1 - m["p1 pr win opp adjust mix"])), axis=1)
compute_lloss["lloss no sos adjust"] = compute_lloss.apply(lambda m: (m["winner"]*np.log(m["p1 pr win beta reg no sos adjust mix"]) + (1-m["winner"])*np.log(1 - m["p1 pr win beta reg no sos adjust mix"])), axis=1)
N = compute_lloss.shape[0]

lloss_sos_adjust = (compute_lloss["lloss sos adjust"].sum()) * (-1/N)
lloss_no_sos_adjust = (compute_lloss["lloss no sos adjust"].sum()) * (-1/N)

print(f"Sos adjust log loss: {lloss_sos_adjust}")
print(f"No sos adjust log loss: {lloss_no_sos_adjust}")
print(f"Random guessing log loss: {-np.log(0.5)}")

Sos adjust log loss: 0.6031435710435108
No sos adjust log loss: 0.6108105575430793
Random guessing log loss: 0.6931471805599453


/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/Users/VillasBoas/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [195]:
temp = [0.5,0.8,0.7]+[0.5,0.8,0.7]
temp2 = [0.25,0.4,0.6]+[0.25,0.4,0.6]

beta.fit(temp, floc=0, fscale=1)

(9.616623800843021, 4.822610250473773, 0, 1)

In [196]:
beta.fit(temp2, floc=0, fscale=1)

(4.839555058182907, 6.765835039698072, 0, 1)